In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [947 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

--2022-10-04 03:25:26--  https://jdbc.postgresql.org/download/postgresql-42.5.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1046274 (1022K) [application/java-archive]
Saving to: ‘postgresql-42.5.0.jar’

postgresql-42.5.0.j 100%[===================>]   1022K  6.20MB/s    in 0.2s    

2022-10-04 03:25:26 (6.20 MB/s) - ‘postgresql-42.5.0.jar’ saved [1046274/1046274]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.5.0.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://dilnigardolkun.s3.us-west-2.amazonaws.com/amazon_reviews_us_Video_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49033728|R1P1G5KZ05H6RD|6302503213|     748506413|The Night They Sa...|           Video|          5|            0|          0|   N|                Y|    Very satisfied!!|Fast shipping. Pl...| 2015-08-31|
|         US|   17857748|R106N066IUN8ZV|B000059PET|     478710180|Hamlet / Kline, N...|           Video|          5|    

In [10]:
# Create a new dataframe with votes 20 or more.
votes_df = df.filter("total_votes >=20")
votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2134748| RN8GRZ45FA27G|B00004RLLT|     401019781|        Shakma [VHS]|           Video|          5|           30|         31|   N|                N|It was some of th...|I starred in this...| 2015-05-22|
|         US|    1750370|R3DHJNHOZTXZ6C|6304457316|     114967273|    Stagecoach [VHS]|           Video|          1|    

In [24]:
# Create a new dataframe with the dataframe from the previous step.
votes_percentage_df = votes_df.filter("helpful_votes/total_votes >= 0.5")
votes_percentage_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2134748| RN8GRZ45FA27G|B00004RLLT|     401019781|        Shakma [VHS]|           Video|          5|           30|         31|   N|                N|It was some of th...|I starred in this...| 2015-05-22|
|         US|    6227904|R2BVILHJN8OCJB|B00004XPP0|     745272701|Star Wars Trilogy...|           Video|          5|    

In [32]:
paid_vine_program_df = votes_percentage_df.filter(votes_percentage_df["vine"] == "Y")
paid_vine_program_df.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [33]:
unpaid_vine_program_df = votes_percentage_df.filter(votes_percentage_df["vine"] == "N")
unpaid_vine_program_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2134748| RN8GRZ45FA27G|B00004RLLT|     401019781|        Shakma [VHS]|           Video|          5|           30|         31|   N|                N|It was some of th...|I starred in this...| 2015-05-22|
|         US|    6227904|R2BVILHJN8OCJB|B00004XPP0|     745272701|Star Wars Trilogy...|           Video|          5|    

In [34]:
paid_total_review = paid_vine_program_df.count()
paid_total_review

0

In [35]:
unpaid_total_review = unpaid_vine_program_df.count()
unpaid_total_review

20201

In [37]:
paid_five_star_review = paid_vine_program_df.filter("star_rating == 5").count()
paid_five_star_review 

0

In [38]:
unpaid_five_star_review = unpaid_vine_program_df.filter("star_rating == 5").count()
unpaid_five_star_review 

12331

In [47]:
#calculating the percentage of 5-star reviews for the paid vine program. Since there is no paid vine program in this dataset, and zero division error, I included "try" & "except" in my code.
try:
    paid_five_star_review / paid_total_review * 100
except ZeroDivisionError:
    paid_five_star_review_percentage= 0
paid_five_star_review_percentage

0

In [44]:
unpaid_five_star_review_percentage = unpaid_five_star_review / unpaid_total_review * 100
unpaid_five_star_review_percentage

61.04153259739616